In [1]:
# import necessary libraries
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.model_selection import train_test_split
import time

In [2]:
#dictionary with usernames their watched web series and ratings out of 5

dataset={
        'Tanya': {'Special Ops': 5,
                  'Criminal Justice': 3,
                  'Panchayat': 3,
                  'Sacred Games': 3,
                  'Apharan': 2,
                  'Mirzapur': 3},
    
        'Mohan': {'Special Ops': 5,
                    'Criminal Justice': 3,
                    'Sacred Games': 5,
                    'Panchayat':5,
                    'Mirzapur': 3,
                    'Apharan': 3},
    
        'Tasha': {'Special Ops': 2,
                   'Panchayat': 5,
                   'Sacred Games': 3,
                   'Mirzapur': 4},
    
        'Nirbhay': {
                   'Mirzapur': 4,
                   'Sacred Games': 4,},
    
       'Muskan': {'Special Ops': 4,
                    'Criminal Justice': 4,
                    'Panchayat': 4,
                    'Mirzapur': 3,
                    'Apharan': 2},
    
       'Anshika': {'Special Ops': 3,
                  'Panchayat': 4,
                  'Mirzapur': 3,
                  'Sacred Games': 5,
                  'Apharan': 3},
    
      'Dhawal': {'Panchayat':4,
                  'Apharan':1,
                  'Sacred Games':4}}

In [3]:
#create a data frame of this dataset

dataset_df=pd.DataFrame(dataset)
dataset_df.fillna("Not Seen Yet",inplace=True)
dataset_df

,Tanya,Mohan,Tasha,Nirbhay,Muskan,Anshika,Dhawal
Special Ops,5,5,2.0,Not Seen Yet,4.0,3.0,Not Seen Yet
Criminal Justice,3,3,Not Seen Yet,Not Seen Yet,4.0,Not Seen Yet,Not Seen Yet
Panchayat,3,5,5.0,Not Seen Yet,4.0,4.0,4.0
Sacred Games,3,5,3.0,4.0,Not Seen Yet,5.0,4.0
Apharan,2,3,Not Seen Yet,Not Seen Yet,2.0,3.0,1.0
Mirzapur,3,3,4.0,4.0,3.0,3.0,Not Seen Yet


In [4]:
# custom function to create unique set of web series

def unique_items():
    unique_items_list = []
    for person in dataset.keys():
        for items in dataset[person]:
            unique_items_list.append(items)
    s=set(unique_items_list)
    unique_items_list=list(s)
    return unique_items_list

In [5]:
# custom function to create cosine correlation method from scratch

def item_similarity(item1,item2):
    both_rated = {}
    
    # Mean of each item
    item1_ratings = [dataset[person][item1] for person in dataset if item1 in dataset[person]]
    item1_mean = sum(item1_ratings) / len(item1_ratings)
    
    item2_ratings = [dataset[person][item2] for person in dataset if item2 in dataset[person]]
    item2_mean = sum(item2_ratings) / len(item2_ratings)
    
    for person in dataset.keys():
        if item1 in dataset[person] and item2 in dataset[person]:
            both_rated[person] = [dataset[person][item1],dataset[person][item2]]

    number_of_ratings = len(both_rated)
    if number_of_ratings == 0:
        return 0
    
    # Sum up the squares of preferences of each user
    item1_square_preferences_sum = sum([pow(value[0]-item1_mean, 2) for key,value in both_rated.items()])
    item2_square_preferences_sum = sum([pow(value[1]-item2_mean, 2) for key,value in both_rated.items()])

    # Sum up the product value of both preferences for each item
    product_sum_of_both_users = sum([(value[0]-item1_mean) * (value[1]-item2_mean) for key,value in both_rated.items()])

    # Calculate the pearson score
    denominator_value = sqrt(item1_square_preferences_sum)*sqrt(item2_square_preferences_sum)
    if denominator_value == 0:
        return 0
    else:
        r = product_sum_of_both_users / denominator_value
        return r

In [6]:
#custom function to filter the seen movies and unseen movies of the target user

def target_movies_to_users(target_person):
    target_person_movie_lst = []
    unique_list =unique_items()
    for movies in dataset[target_person]:
        target_person_movie_lst.append(movies)

    s=set(unique_list)
    recommended_movies=list(s.difference(target_person_movie_lst))
    a = len(recommended_movies)
    if a == 0:
        return 0
    return recommended_movies,target_person_movie_lst

In [7]:
#custom function to check most similar items 

def most_similar_items(target_item):
    un_lst=unique_items()
    scores = [(item_similarity(target_item,other_item),target_item+" --> "+other_item) for other_item in un_lst if other_item!=target_item]
    scores.sort(reverse=True)
    similar_items = [[score,item] for score, item in scores]
    
    return similar_items

In [8]:
def recommendation_phase(target_person,target_movie):
    if target_movies_to_users(target_person=target_person) == 0:
        print(target_person, "has seen all the movies")
        return -1
    not_seen_movies,seen_movies=target_movies_to_users(target_person=target_person)
    seen_ratings = [[dataset[target_person][movies],movies] for movies in dataset[target_person]]
    weighted_avg,weighted_sim = 0,0
    for rate,movie in seen_ratings:
        item_sim=item_similarity(target_movie,movie)
        weighted_avg +=(item_sim*rate)
        weighted_sim +=item_sim
    weighted_rank=weighted_avg/weighted_sim

    return weighted_rank

In [9]:
recommendation_phase('Nirbhay','Panchayat')

4.0